In [35]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import nibabel as nib

from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import rand_score
from nibabel.nifti1 import Nifti1Image


def load_data(hemisphere):
    X = pd.read_csv('https://raw.githubusercontent.com/JonahKember/hippocampus_clustering/main/' + hemisphere + '_nmf_input_raw.csv')
    X = X.to_numpy()
    return(X)


def get_age(hemisphere):
    Y = pd.read_csv(hemisphere + '_age.csv')
    age = Y['Age'].to_numpy()
    return(age)


def run_nmf(X,k):
    
    '''1. Rescale columns of X to the interval [0,1].
    2. Run Non-Negative matrix factorization on X with k components.'''

    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    model = NMF(n_components = k, init = 'random', max_iter = 10000)
    model.fit_transform(X)
    
    W = model.transform(X)
    clusters = np.argmax(W,1)

    return(clusters, model)

def get_nmf_similarity(X, k, n_shuffle = 5):
    
    '''Calculate the stability of NMF components. 
    1. Randomly split the matrix X in half n_shuffle times.
    2. Run NMF on each half (with k components).
    3. Calculate the adjusted Rand index between the clusters output from each half.'''

    rand_indices = np.zeros(n_shuffle)

    for split in range(n_shuffle):

        # Generate randomly shuffled indices (seed set for reproducibility).
        np.random.seed(split)
        idx = np.array_split (np.random.permutation(X.shape[1]),2)

        # Get NMF clusters for each split.
        clust_1 = run_nmf(X[:,idx[0]],k)[0]
        clust_2 = run_nmf(X[:,idx[1]],k)[0]

        # Calculate the adjusted Rand index.
        rand_indices[split] = adjusted_rand_score(clust_1,clust_2)
        
    print('Similarity for k = '+ str(k) + ' is ' + str(np.mean(rand_indices)))
    
    return(rand_indices)

def get_nmf_error(X, k_vals = np.arange(1,10)):
    
    '''Calculate the reconstruction error of NMF components.''' 

    error = np.zeros(len(k_vals) + 1)

    for k in k_vals:

        # Get NMF clusters for each split.
        model = run_nmf(X,k)[1]
        error[k] = model.reconstruction_err_
        
        print('Reconstruction error for k = '+ str(k) + ' is ' + str(np.mean(error[k])))
    
    return(error)


def clusters_to_nii(k, hemisphere,clusters):

    if hemisphere == 'right':
        hem_label = 1
    elif hemisphere == 'left':
        hem_label = 2

    # Open majority-vote label.
    majority_vote = nib.load('majority_vote_label.nii')

    # Get nifti file information.
    labels = majority_vote.get_fdata()

    # Initialize empty matrix, fill with cluster labels, write to nifti.
    nii = np.zeros(labels.shape, dtype = np.float64)
    nii[labels == hem_label] = clusters
    nii = Nifti1Image(nii, affine = majority_vote.affine, header = majority_vote.header)

    nib.save(nii, str(hemisphere + '_nmf_clusters_k' + str(k) + '.nii.gz'))


In [36]:
def load_data_adults(hemisphere,age_threshold):

    X = load_data(hemisphere)
    age = get_age(hemisphere)

    idx = (age > age_threshold)

    X = X[:,np.concatenate((idx,idx,idx))]
    return(X)


In [37]:
# Number of data-set splits.
n_shuffle = 5

# Number of k-values to explore.
n_k = 15

X = load_data_adults('right',16)

# Track similarity of NMF components across range of k-values.
k_similarity = np.zeros([n_k,n_shuffle])
for k in range(1,n_k):
    k_similarity[k,:] = get_nmf_similarity(X, k, n_shuffle = n_shuffle)
    


Similarity for k = 1 is 1.0
Similarity for k = 2 is 0.4029005640572717
Similarity for k = 3 is 0.4828470446924924
Similarity for k = 4 is 0.35566823736872377
Similarity for k = 5 is 0.37828510502615403
Similarity for k = 6 is 0.37044734933034174
Similarity for k = 7 is 0.39529619692759427
Similarity for k = 8 is 0.2713622014117951
Similarity for k = 9 is 0.3328772042587498
Similarity for k = 10 is 0.29675547120251433
Similarity for k = 11 is 0.1930869212142759
Similarity for k = 12 is 0.1934104723621019
Similarity for k = 13 is 0.1920966081758651
Similarity for k = 14 is 0.2742498777732353


In [ ]:
# Number of k-values to explore.
k_vals = np.arange(1,15)
X = load_data('right')

error = get_nmf_error(X, k_vals = k_vals)

In [ ]:
# Plot the similarity.
x = np.arange(k_similarity.shape[0])
y = np.mean(k_similarity,1)
yerr = np.std(k_similarity, axis = 1)/np.sqrt(k_similarity.shape[1])

fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].errorbar(x[1:],y[1:], yerr = yerr[1:])

ax[0].set_xlim([0,15])
ax[0].set_xlabel('k')
ax[0].set_ylabel('Adjusted Rand Index')

# Plot the reconstruction error.
x = np.arange(error.shape[0])
ax[1].plot(x[1:],error[1:])

ax[1].set_xlim([0,15])
ax[1].set_xlabel('k')
ax[1].set_ylabel('Reconstruction error')

In [ ]:
# Load subfield labels.
y = pd.read_csv('right_subfield_labels.csv')
y = y.to_numpy()
y = y.reshape(y.shape[0],)

clusters = run_nmf(X,5)[0]

model = LinearDiscriminantAnalysis(n_components = 2)
model.fit_transform(X,clusters)
lda_components = model.transform(X)

# Plot Subfields against NMF components in LDA space. 
fig, ax = plt.subplots(1,2,figsize = [15,5])

for k in range(np.max(clusters) + 1):
    ax[0].plot(lda_components[y == k,0], lda_components[y == k,1], marker='o', linestyle='')    
    ax[1].plot(lda_components[clusters == k,0], lda_components[clusters == k,1], marker='o', linestyle='') 

plt.show()


In [ ]:
majority_vote = nib.load('majority_vote_label.nii')
hcpd = nib.load('right_components_k5.nii')
hcp = nib.load('raihann_warped.nii.gz')


labels = majority_vote.get_fdata()

hcpd = hcpd.get_fdata()
hcpd = hcpd[labels == 1]

hcp = hcp.get_fdata()
hcp = hcp[labels == 1]
hcp = np.round(hcp)

hcpd = hcpd.astype(int)
hcp = hcp.astype(int)

print(hcpd)
print(hcp)
adjusted_rand_score(hcpd,hcp)